In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment.
# For the code to do so you can search “mount” in code snippets or use the code given below.
# Our entire drive contents are now mounted on colab at the location “/gdrive”.
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!pip install vecstack

In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter #for Smote,

import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

In [ ]:
trainfile = r'/gdrive/My Drive/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)




testfile = r'/gdrive/My Drive/RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)




print(train_data.shape)
print(test_data.shape)
print(train_data.head())

(65000, 596)
(173836, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16  

In [ ]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Columns: 596 entries, CoverageField11A to QuoteConversion_Flag
dtypes: float64(5), int64(591)
memory usage: 295.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173836 entries, 0 to 173835
Columns: 596 entries, CoverageField11A to GeographicField64
dtypes: float64(5), int64(590), object(1)
memory usage: 790.5+ MB


In [ ]:
# To check number of null values
train_data.isnull().sum().sort_values(ascending=False)

CoverageField11A        0
PersonalField17_YU      0
PersonalField17_YW      0
PersonalField17_YX      0
PersonalField17_YY      0
                       ..
PersonalField64         0
PersonalField65         0
PersonalField66         0
PersonalField67         0
QuoteConversion_Flag    0
Length: 596, dtype: int64

In [ ]:
print(train_data.select_dtypes(include=['object']))

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[65000 rows x 0 columns]


In [ ]:
print(test_data.select_dtypes(include=['object']))

       GeographicField64
0                     IL
1                     NJ
2                     NJ
3                     TX
4                     CA
...                  ...
173831                NJ
173832                CA
173833                CA
173834                TX
173835                NJ

[173836 rows x 1 columns]


In [ ]:
# drop categorical data
test_data.drop('GeographicField64',axis=1,inplace=True)

In [ ]:
#seperate features and target variable in training data
X=train_data.drop('QuoteConversion_Flag',axis=1)
y=train_data['QuoteConversion_Flag']
smote = SMOTE(random_state=42, sampling_strategy=0.75)
X_res, y_res = smote.fit_resample(X, y)

In [ ]:
X_res.shape

(79107, 595)

In [ ]:
#Re-adding target variable to feature set for corr calculation
res_data=pd.DataFrame(X_res, columns=X.columns)
res_data['QuoteConversion_Flag']=y_res
#Feature selection with QuoteConversion_Flag
corr=res_data.corr()['QuoteConversion_Flag'].abs().sort_values(ascending=False)
top_150=corr.index[1:151]
X_resselect=res_data[top_150]

In [ ]:
X_resselect.shape


(79107, 150)

In [ ]:
y_res.shape

(79107,)

In [ ]:
#select same feature from test data
test_dataselec=test_data[top_150]

In [ ]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]

S_Train, S_Test = stacking(models,
                           X_resselect, y_res, test_dataselec,
                           regression=False,
                           mode='oof_pred_bag',
                           needs_proba=False,
                           save_dir=None,
                           metric=accuracy_score,
                           n_folds=4,
                           stratified=True,
                           shuffle=True,
                           random_state=0,
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.80072812]
    fold  1:  [0.79926177]
    fold  2:  [0.79243566]
    fold  3:  [0.79601537]
    ----
    MEAN:     [0.79711023] + [0.00319251]
    FULL:     [0.79711024]

model  1:     [MLPClassifier]
    fold  0:  [0.90711432]
    fold  1:  [0.91166507]
    fold  2:  [0.91394044]
    fold  3:  [0.91044701]
    ----
    MEAN:     [0.91079171] + [0.00246573]
    FULL:     [0.91079171]

model  2:     [LinearSVC]
    fold  0:  [0.74915306]
    fold  1:  [0.83162259]
    fold  2:  [0.85771351]
    fold  3:  [0.60366100]
    ----
    MEAN:     [0.76053754] + [0.09904053]
    FULL:     [0.76053952]

model  3:     [RandomForestCla

In [ ]:
mlp = MLPClassifier(random_state=42)

# Define hyperparameters to search
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(mlp, param_grid, cv=3, n_jobs=-1)
grid_search.fit(S_Train, y_res)

# Get the best hyperparameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
#use stacked data to train the model
mlp=MLPClassifier(max_iter=80)
mlp.fit(S_Train, y_res)
mlp_predict=mlp.predict(S_Test)


In [ ]:
#predict MLP
predicted_prob=mlp.predict_proba(S_Test)[:,1]

In [ ]:
submission_df = pd.DataFrame({
    'QuoteNumber': test_data['QuoteNumber'],
     'QuoteConversion_Flag': predicted_prob
})
submission_df.to_csv('/gdrive/My Drive/revise1.csv', index=False)

In [ ]:
smote = SMOTE(random_state=42, sampling_strategy=0.5)
X_res, y_res = smote.fit_resample(X, y)

In [ ]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]

S_Train, S_Test = stacking(models,
                           X_resselect, y_res, test_dataselec,
                           regression=False,
                           mode='oof_pred_bag',
                           needs_proba=False,
                           save_dir=None,
                           metric=accuracy_score,
                           n_folds=4,
                           stratified=True,
                           shuffle=True,
                           random_state=0,
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.80072812]
    fold  1:  [0.79926177]
    fold  2:  [0.79243566]
    fold  3:  [0.79601537]
    ----
    MEAN:     [0.79711023] + [0.00319251]
    FULL:     [0.79711024]

model  1:     [MLPClassifier]
    fold  0:  [0.90676038]
    fold  1:  [0.83268443]
    fold  2:  [0.88800121]
    fold  3:  [0.91418892]
    ----
    MEAN:     [0.88540873] + [0.03190127]
    FULL:     [0.88540837]

model  2:     [LinearSVC]
    fold  0:  [0.90397937]
    fold  1:  [0.86226425]
    fold  2:  [0.90180513]
    fold  3:  [0.85694782]
    ----
    MEAN:     [0.88124914] + [0.02173817]
    FULL:     [0.88124945]

model  3:     [RandomForestCla

In [ ]:
mlp = MLPClassifier(random_state=42)

# Define hyperparameters to search
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(mlp, param_grid, cv=3, n_jobs=-1)
grid_search.fit(S_Train, y_res)

# Get the best hyperparameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
#use stacked data to train the model
mlp=MLPClassifier(max_iter=80)
mlp.fit(S_Train, y_res)
mlp_predict=mlp.predict(S_Test)


In [ ]:
predicted_prob=mlp.predict_proba(S_Test)[:,1]

In [ ]:
submission_df = pd.DataFrame({
    'QuoteNumber': test_data['QuoteNumber'],
     'QuoteConversion_Flag': predicted_prob
})
submission_df.to_csv('/gdrive/My Drive/revise1.csv', index=False)